In [3]:
import sqlite3
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import entropy
from numpy.linalg import matrix_rank
import os

# === USER CONFIGURATION ===
DB_PATHS = [
    "/Users/......./pso_r_7i.db",
    "/Users/......./pso_Qgauss_2.db",
    "/Users/......./pso_Qcauchy_1.db",
    "/Users/......./pso_QLevyII_1.db"
    # Add more paths here if needed
]
PARAM_COLS = ["loadb", "Es", "rhos", "nus", "cs", "phis",
              "epss", "thetass", "hks", "H1", "H2", "x6"]
MIN_BIN_THRESHOLD = 15
N_BINS = 5
OUTPUT_CSV = "s2_sampling_diagnostics_merged.csv"

# === MERGE DATA FROM MULTIPLE DATABASES ===
all_data = []
for path in DB_PATHS:
    if not os.path.exists(path):
        print(f"⚠️ Skipping missing file: {path}")
        continue
    print(f"✅ Loading: {path}")
    conn = sqlite3.connect(path)
    query = f"SELECT {', '.join(PARAM_COLS)}, e_plst FROM particles"
    try:
        df_part = pd.read_sql_query(query, conn)
        all_data.append(df_part)
    except Exception as e:
        print(f"⚠️ Failed to read from {path}: {e}")
    finally:
        conn.close()

# Merge all data
if not all_data:
    raise ValueError("❌ No valid data loaded from the listed databases.")
df = pd.concat(all_data, ignore_index=True)

# === CLEAN DATA ===
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(subset=["e_plst"], inplace=True)

# === SAMPLING DIAGNOSTICS ===
sufficiency_diagnostics = []

for i, j in combinations(PARAM_COLS, 2):
    try:
        df["bin_i"] = pd.qcut(df[i], q=N_BINS, duplicates="drop")
        df["bin_j"] = pd.qcut(df[j], q=N_BINS, duplicates="drop")

        bin_counts = df.groupby(["bin_i", "bin_j"], observed=True).size().unstack(fill_value=0)
        mean_ppi_grid = df.groupby(["bin_i", "bin_j"], observed=True)["e_plst"].mean().unstack(fill_value=np.nan)

        count_values = bin_counts.values.flatten()
        p = count_values[count_values > 0] / count_values.sum()

        populated_bins = (count_values > 0).sum()
        total_bins = bin_counts.size
        bin_entropy = entropy(p)
        bin_gini = 1 - np.sum(p ** 2)
        avg_samples = np.mean(count_values[count_values > 0])
        min_samples = np.min(count_values[count_values > 0])
        rank = matrix_rank(mean_ppi_grid.fillna(mean_ppi_grid.mean()).values)

        sufficiency_diagnostics.append({
            "Parameter 1": i,
            "Parameter 2": j,
            "Populated Bins": populated_bins,
            "Total Bins": total_bins,
            "Entropy": bin_entropy,
            "Gini Index": bin_gini,
            "Avg Samples/Bin": avg_samples,
            "Min Samples/Bin": min_samples,
            "PPI Grid Rank": rank,
            "Sufficient": populated_bins >= MIN_BIN_THRESHOLD
        })
    except Exception as e:
        print(f"⚠️ Skipping {i}-{j}: {e}")
        continue

# === OUTPUT ===
df_out = pd.DataFrame(sufficiency_diagnostics)
df_out.sort_values(by="Populated Bins", ascending=False, inplace=True)
df_out.to_csv(OUTPUT_CSV, index=False)

print(f"\n✅ Sampling diagnostics complete across {len(DB_PATHS)} databases.")
print(f"📄 Results saved to: {OUTPUT_CSV}")
print(df_out.head(10))


✅ Loading: /Users/israr/Documents/2025/LSwarm/pso_r_7i.db
✅ Loading: /Users/israr/Documents/2025/LSwarm/pso_Qgauss_2.db
✅ Loading: /Users/israr/Documents/2025/LSwarm/pso_Qcauchy_1.db
✅ Loading: /Users/israr/Documents/2025/LSwarm/pso_QLevyII_1.db

✅ Sampling diagnostics complete across 4 databases.
📄 Results saved to: s2_sampling_diagnostics_merged.csv
   Parameter 1 Parameter 2  Populated Bins  Total Bins   Entropy  Gini Index  \
22        rhos          cs              25          25  3.100838    0.950664   
43          cs          H2              25          25  3.069214    0.947349   
28        rhos          H2              25          25  3.022123    0.943577   
49        phis          H2              20          20  2.801119    0.928010   
24        rhos        epss              20          20  2.776625    0.926216   
25        rhos     thetass              20          20  2.766834    0.924574   
26        rhos         hks              20          20  2.774837    0.925718   
27    

In [7]:
import sqlite3
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import entropy
from numpy.linalg import matrix_rank
from scipy.spatial.distance import pdist

# === USER CONFIGURATION ===
DB_PATHS = [
    "/Users/......./pso_r_7_iter1_200.db",
    "/Users/......./pso_Qgauss_2.db",
    "/Users/......./pso_Qcauchy_1.db",
    "/Users/......./pso_QLevyII_1.db"
    # Add more paths here if needed
]
# DB_PATHS = [
#     "/Users/israr/Documents/2025/LSwarm/pso_r_7_iter1_200.db"
#     # Add more paths here if needed
# ]
PARAM_COLS = ["loadb", "Es", "rhos", "nus", "cs", "phis",
              "epss", "thetass", "hks", "H1", "H2", "x6"]
MIN_BIN_THRESHOLD = 15
N_BINS = 5
OUTPUT_CSV = "s2_sampling_diagnostics_merged.csv"

# === MERGE DATA FROM MULTIPLE DATABASES ===
all_data = []
for path in DB_PATHS:
    if not os.path.exists(path):
        print(f"⚠️ Skipping missing file: {path}")
        continue
    print(f"✅ Loading: {path}")
    conn = sqlite3.connect(path)
    query = f"SELECT {', '.join(PARAM_COLS)}, ppi FROM particles"
    try:
        df_part = pd.read_sql_query(query, conn)
        all_data.append(df_part)
    except Exception as e:
        print(f"⚠️ Failed to read from {path}: {e}")
    finally:
        conn.close()

# Merge all data
if not all_data:
    raise ValueError("❌ No valid data loaded from the listed databases.")
df = pd.concat(all_data, ignore_index=True)

# === CLEAN DATA ===
# CLEAN CORRECTLY
df_clean = df.copy()
df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
df_clean.dropna(subset=["ppi"], inplace=True)

# === GLOBAL DIAGNOSTICS ===
print("📊 Calculating global sampling diagnostics...")
total_samples = len(df_clean)
unique_samples = df_clean[PARAM_COLS].drop_duplicates().shape[0]
redundancy_ratio = 1 - unique_samples / total_samples
average_pairwise_distance = pdist(df_clean[PARAM_COLS].values).mean()
unique_per_param = df_clean[PARAM_COLS].nunique().to_dict()

print("✅ Global Stats:")
print(f" - Total Samples: {total_samples}")
print(f" - Unique Samples: {unique_samples}")
print(f" - Redundancy Ratio: {redundancy_ratio:.4f}")
print(f" - Avg Pairwise Distance: {average_pairwise_distance:.4f}")

# === LOCAL DIAGNOSTICS PER PARAMETER PAIR ===
print("📊 Running S2 bin coverage diagnostics...")
sufficiency_diagnostics = []

for i, j in combinations(PARAM_COLS, 2):
    try:
        df_clean["bin_i"] = pd.qcut(df_clean[i], q=N_BINS, duplicates="drop")
        df_clean["bin_j"] = pd.qcut(df_clean[j], q=N_BINS, duplicates="drop")

        bin_counts = df_clean.groupby(["bin_i", "bin_j"], observed=True).size().unstack(fill_value=0)
        mean_ppi_grid = df_clean.groupby(["bin_i", "bin_j"], observed=True)["ppi"].mean().unstack(fill_value=np.nan)

        count_values = bin_counts.values.flatten()
        p = count_values[count_values > 0] / count_values.sum()

        populated_bins = (count_values > 0).sum()
        total_bins = bin_counts.size
        bin_entropy = entropy(p)
        bin_gini = 1 - np.sum(p ** 2)
        avg_samples = np.mean(count_values[count_values > 0])
        min_samples = np.min(count_values[count_values > 0])
        rank = matrix_rank(mean_ppi_grid.fillna(mean_ppi_grid.mean()).values)

        sufficiency_diagnostics.append({
            "Parameter 1": i,
            "Parameter 2": j,
            "Populated Bins": populated_bins,
            "Total Bins": total_bins,
            "Entropy": bin_entropy,
            "Gini Index": bin_gini,
            "Avg Samples/Bin": avg_samples,
            "Min Samples/Bin": min_samples,
            "PPI Grid Rank": rank,
            "Sufficient": populated_bins >= MIN_BIN_THRESHOLD
        })
    except Exception as e:
        print(f"⚠️ Skipping {i}-{j}: {e}")
        continue

# === SAVE RESULTS ===
df_out = pd.DataFrame(sufficiency_diagnostics)
df_out.sort_values(by="Populated Bins", ascending=False, inplace=True)
df_out.to_csv(OUTPUT_CSV, index=False)

num_false = (~df_out["Sufficient"]).sum()

# === GLOBAL STATUS EVALUATION FUNCTION ===
def evaluate_global_quality(row):
    status = {}
    unique_ratio = unique_samples / total_samples
    status["Total Samples Status"] = (
        "✅ Good" if total_samples >= 1000 else
        "⚠️ Low" if total_samples >= 500 else
        "❌ Very Low"
    )
    status["Unique Samples Status"] = (
        "✅ High" if unique_ratio >= 0.95 else
        "⚠️ Moderate" if unique_ratio >= 0.9 else
        "❌ Low"
    )
    status["Redundancy Status"] = (
        "✅ Low" if redundancy_ratio <= 0.05 else
        "⚠️ Moderate" if redundancy_ratio <= 0.15 else
        "❌ High"
    )
    status["Pairwise Distance Status"] = "ℹ️ Relative — compare across runs"
    for param in PARAM_COLS:
        val = unique_per_param[param]
        status[f"{param} Coverage"] = (
            "✅ Excellent" if val >= 500 else
            "⚠️ Acceptable" if val >= 200 else
            "❌ Sparse"
        )
    return status

# Generate global status
global_status = evaluate_global_quality(df)
global_status["Number of S2 Pairs (False)"] = num_false
global_status["Number of S2 Pairs (Total)"] = len(df_out)
global_status["S2 Sufficiency Coverage (%)"] = 100 * (len(df_out) - num_false) / len(df_out)

# Print results
print("\n=== GLOBAL STATUS EVALUATION ===")
for k, v in global_status.items():
    print(f"{k}: {v}")

df_out = pd.DataFrame(sufficiency_diagnostics)
df_out.sort_values(by="Populated Bins", ascending=False, inplace=True)
df_out.to_csv(OUTPUT_CSV, index=False)

print(f"\n✅ S2 bin diagnostics saved to {OUTPUT_CSV}")

# === Optional: Display Global Summary ===
print("\n=== GLOBAL SUMMARY ===")
print(f"Total Samples: {total_samples}")
print(f"Unique Samples: {unique_samples}")
print(f"Redundancy: {redundancy_ratio:.4f}")
print(f"Avg Pairwise Distance: {average_pairwise_distance:.4f}")
for k, v in unique_per_param.items():
    print(f"Unique values in {k}: {v}")


✅ Loading: /Users/israr/Documents/2025/LSwarm/pso_r_7_iter1_200.db
✅ Loading: /Users/israr/Documents/2025/LSwarm/pso_Qgauss_2.db
✅ Loading: /Users/israr/Documents/2025/LSwarm/pso_Qcauchy_1.db
✅ Loading: /Users/israr/Documents/2025/LSwarm/pso_QLevyII_1.db
📊 Calculating global sampling diagnostics...
✅ Global Stats:
 - Total Samples: 6034
 - Unique Samples: 6012
 - Redundancy Ratio: 0.0036
 - Avg Pairwise Distance: 63138312.4833
📊 Running S2 bin coverage diagnostics...

=== GLOBAL STATUS EVALUATION ===
Total Samples Status: ✅ Good
Unique Samples Status: ✅ High
Redundancy Status: ✅ Low
Pairwise Distance Status: ℹ️ Relative — compare across runs
loadb Coverage: ✅ Excellent
Es Coverage: ✅ Excellent
rhos Coverage: ✅ Excellent
nus Coverage: ✅ Excellent
cs Coverage: ✅ Excellent
phis Coverage: ✅ Excellent
epss Coverage: ✅ Excellent
thetass Coverage: ✅ Excellent
hks Coverage: ✅ Excellent
H1 Coverage: ✅ Excellent
H2 Coverage: ✅ Excellent
x6 Coverage: ✅ Excellent
Number of S2 Pairs (False): 0
Numb